In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from itertools import islice


In [4]:
# Reading in file and adding it to array to one hot encode. 
#file = open('samplemusic.txt')
file = open('train.txt')
tag = False
part_tag = False
array = []
tag_string = ''
while 1:
    char = file.read(1)  # read by character
    if not char: break
        
    # if char is beginning of tag
    if char == '<':
        part_tag = True
        tag_string += char
        
    # if char is end of tag
    elif char == '>':
        # Checks if > is part of <start>/<end> tags or lone char
        if tag_string == '<start' or tag_string == '<end':
            tag = False
            tag_string += char
            array.append(tag_string)
        # > is not part of tag, just lone char so we append it and reset the tag_string
        else: 
            array.append(char)
        tag_string = ''
        
    
    # checks if char is still part of tag
    elif part_tag == True:
        # If next char after < is either s or e, we assume this is a tag.
        if char == 's' or char == 'e':
            tag = True
            tag_string += char
        # If not, < is just a lone char so append both separately
        else:
            array.append(tag_string)
            array.append(char)
            tag_string = ''
        part_tag = False
        
    # If inside tag, keep adding chars to create full tag
    elif tag == True:
        tag_string += char
            
    # if char is not part of tag, just add char
    elif tag == False:
        array.append(char)
    
file.close()

In [5]:
# One-hot encoding of data
array = np.array(array).reshape(-1,1)
print('Shape or original data array:', array.shape)
print('Number of unique chars in array:', len(np.unique(array)))
onehot_encoder = OneHotEncoder(sparse=False)
X_data = onehot_encoder.fit_transform(array) # Contains One hot encoding of all chars in text 

# Example of inversing one hot to get letter
inverted = onehot_encoder.inverse_transform(X_data[0, :].reshape(1,-1))
print(inverted)
print('Shape of one-hot encoded data, X_data:', X_data.shape)

Shape or original data array: (185410, 1)
Number of unique chars in array: 153
[['<start>']]
Shape of one-hot encoded data, X_data: (185410, 153)


In [27]:
# Turning songs from data into chunks 
train_data = []
song = []
temp = []
for idx in range(len(X_data)):
    temp.append(X_data[idx]) # Add 100 chars into temp to create chunks 
    chara = onehot_encoder.inverse_transform(X_data[idx, :].reshape(1,-1))[0][0]
    # Checks if char is at the end of a song 
    #print(chara)
    if chara == '<end>':
        song.append(temp)
        train_data.append(song)
        temp = []
        song = []
    
    # If 100 chars in temp, add chunk to train_data
    if idx%100 == 99:
        song.append(temp)
        temp = []

train_data = np.array(train_data)
print(train_data.shape[0])

333


In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)

    def forward(self, x, hidden):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True

        # Reshape input
        x.view(x.size(0), self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)

        out, hidden = self.lstm(x, hidden)
        return out.view(-1, num_classes), hidden
    
    def init_hidden(self):
        if torch.cuda.is_available():
            hidden = torch.zeros(self.n_layers, 1, self.hidden_size).cuda()
        else:
            hidden = torch.zeros(self.n_layers, 1, self.hidden_size)

        return Variable(hidden)

In [ ]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

In [ ]:
torch.manual_seed(777)  # reproducibility
num_classes = train_data.shape[1]
input_size = train_data.shape[1]  # one-hot size
hidden_size = train_data.shape[1]  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 1  # One hundred sized chunks of text 
num_layers = 1  # one-layer rnn

# Instantiate RNN model
lstm = LSTM(num_classes, input_size, hidden_size, num_layers).to(device)
print(rnn)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.1)

total_loss = []
avg_minibatch_loss = []
N = 50
# Train the model
for epoch in range(2):
    N_chunk_loss = 0.0
    correct = 0
    total = 0
    for song in range(len(train_data)):
        optimizer.zero_grad()
        for chunk in range(len(song)):
            targets = np.roll(chunk, -1).to(device) # Targets is the chunk left shifted once
            hidden = lstm.init_hidden() # Inits hidden state
            loss = 0
            for c in range(len(input)):
                output, hidden = lstm(chunk[c], hidden)
                loss += criterion(output, target[c])

            loss.backward()
            optimizer.step()
            N_chunk_loss += loss
    
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))

print("Learning finished!")